note: all this is right now are a bunch of code snippets that will most likley be valuable to our algo that will need to be modefied 

In [ ]:
import os
from datetime import datetime

import numpy as np
import torch
import wandb
from tqdm import trange


#rllib with ray is one possible candidate for the primary framework for the imitation learning 
from ray import train, tune

#RL option after imitation learning is done
from agilerl.algorithms.ppo import PPO
from agilerl.training.train_on_policy import train_on_policy
from agilerl.utils.utils import create_population, make_skill_vect_envs, make_vect_envs
from agilerl.wrappers.learning import Skill

from ray.rllib.algorithms.bc import BCConfig

observation space: images, actions from other agents

action space: expending activity units, sending signal observable by other agent that also expends an activity unit

expert data: input: images, output: rdm


training stage: use rllib for imitation learning

save model in .pt after training

prediction stage: use agilerl for prediction


In [ ]:
#BC here stands for behavior cloning 

config = BCConfig().training(lr=0.00001, gamma=0.99)
config = config.offline_data(
    input_="./rllib/tests/data/cartpole/large.json")



In [ ]:
    config = (
        PPOConfig()
        .environment(TwoStepGame)
        .framework(args.framework)
        .env_runners(
            batch_mode="complete_episodes",
            num_env_runners=0,
            # TODO(avnishn) make a new example compatible w connectors.
            enable_connectors=False,
        )
        .callbacks(FillInActions)
        .training(model={"custom_model": "cc_model"})
        .multi_agent(
            policies={
                "pol1": (None, observer_space, action_space, {}),
                "pol2": (None, observer_space, action_space, {}),
            },
            policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: "pol1"
            if agent_id == 0
            else "pol2",
            observation_fn=central_critic_observer,
        )
        # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    )

In [ ]:
from ray.rllib.algorithms.marwil import MARWILConfig
# Run this from the ray directory root.
config = MARWILConfig()  
config = config.training(beta=1.0, lr=0.00001, gamma=0.99)  
config = config.offline_data(  
    input_=["./rllib/tests/data/cartpole/large.json"])
print(config.to_dict()) 
"""


"""
# Build an Algorithm object from the config and run 1 training iteration.
algo = config.build()  
algo.train() 